In [6]:
# Recupération de la liste des théâtres et Opéras de paris via Wikipédia

In [7]:
!pip install -q lxml

import bs4
import lxml
import pandas as pd
import urllib

from urllib import request

url_liste_etablissements = "https://fr.wikipedia.org/wiki/Liste_des_th%C3%A9%C3%A2tres_et_op%C3%A9ras_de_Paris"
    
request_text = request.urlopen(url_liste_etablissements).read()
type(request_text)

page = bs4.BeautifulSoup(request_text, "lxml")

tableau_etablissements = page.find("table", {"class":"wikitable sortable"})

table_body = tableau_etablissements.find("tbody")
rows = table_body.find_all("tr")

for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]


dico_etablissements = dict()
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    if len(cols) > 0 : 
        dico_etablissements[cols[0]] = cols[1:]

data_etablissements = pd.DataFrame.from_dict(dico_etablissements,orient='index')
data_etablissements.head()

for row in rows:
    cols = row.find_all('th')
    if len(cols) > 0 : 
        cols = [ele.get_text(separator=' ').strip().title() for ele in cols]
        columns_etablissements = cols

data_etablissements.columns = columns_etablissements[1:]
data_etablissements.head()

,Adresse,Arr.,Date D'Ouverture,Nombre De Sièges,Notes
Théâtre Adyar,"4, square Rapp",7e,1912,381,
Aktéon,"11, rue du Général-Blaise",11e,1986,60,
Antre Magique,"50, rue Saint-Georges",9e,,49,
Artistic Athévains,"45 bis, rue Richard-Lenoir",11e,Début XXe siècle,220,"Ancien nom : L'Artistic, Folies Artistic, Arti..."
A la Folie Théâtre,"6, rue de la Folie-Méricourt",11e,2008,94 et 49,Ancien nom : théâtre de l'Epouventail


In [12]:
import pandas as pd

def obtenir_coordonnees_adresse(adresse):
    base_url = "https://api-adresse.data.gouv.fr/search/"
    params = {
        "q": adresse,
        "limit": 1
    }

    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        data = response.json()
        if data.get('features'):
            coordinates = data['features'][0]['geometry']['coordinates']
            return tuple(coordinates)
        else:
            return None
    else:
        print(f"Erreur lors de la requête : {response.status_code}")
        return None

data_etablissements['Adresse_complete'] = data_etablissements['Adresse'] + ', Paris, France'

data_etablissements['Coordonnees'] = data_etablissements['Adresse_complete'].apply(obtenir_coordonnees_adresse)

print(data_etablissements)


HTTPError: HTTP Error 400: Bad Request

In [10]:
import folium
from folium.plugins import MarkerCluster

data_etablissements['Coordonnees'] = data_etablissements['Adresse_complete'].apply(obtenir_coordonnees_adresse)

carte = folium.Map(location=[48.8566, 2.3522], zoom_start=12)

for index, row in data_etablissements.iterrows():
    if row['Coordonnees']:
        folium.Marker(
            location=row['Coordonnees'],
            popup=row['Adresse_complete']
        ).add_to(marker_cluster)

carte.save("carte_paris.html")


ModuleNotFoundError: No module named 'folium'